For this task I will be using jupyter notebook since it's the best way to showcase my reasoning. I'll try to explain my exact thought process as I would go along with the code. I will be working on n=50000 sample of data due to lack of computer power. 

Let's start by loading data and looking through it. 

<br><br><font size="10"> Preprocessing </font><br><br>

In [457]:
import time 
import pandas as pd
import numpy as np

from datetime import datetime 
from sklearn import preprocessing

In [458]:
DATA_PATH = "Documents/lending-club-loan-data/loan.csv"

In [459]:
def load_and_sample_data(data_path, num_samples=50000):
    
    df = pd.read_csv(data_path, low_memory=False)
    df = df.sample(num_samples)
    
    return df 

In [460]:
df = load_and_sample_data(DATA_PATH)
print('Number of columns: ', len(df.columns), '\nColumn names: ', df.columns.tolist())

Number of columns:  74 
Column names:  ['id', 'member_id', 'loan_amnt', 'funded_amnt', 'funded_amnt_inv', 'term', 'int_rate', 'installment', 'grade', 'sub_grade', 'emp_title', 'emp_length', 'home_ownership', 'annual_inc', 'verification_status', 'issue_d', 'loan_status', 'pymnt_plan', 'url', 'desc', 'purpose', 'title', 'zip_code', 'addr_state', 'dti', 'delinq_2yrs', 'earliest_cr_line', 'inq_last_6mths', 'mths_since_last_delinq', 'mths_since_last_record', 'open_acc', 'pub_rec', 'revol_bal', 'revol_util', 'total_acc', 'initial_list_status', 'out_prncp', 'out_prncp_inv', 'total_pymnt', 'total_pymnt_inv', 'total_rec_prncp', 'total_rec_int', 'total_rec_late_fee', 'recoveries', 'collection_recovery_fee', 'last_pymnt_d', 'last_pymnt_amnt', 'next_pymnt_d', 'last_credit_pull_d', 'collections_12_mths_ex_med', 'mths_since_last_major_derog', 'policy_code', 'application_type', 'annual_inc_joint', 'dti_joint', 'verification_status_joint', 'acc_now_delinq', 'tot_coll_amt', 'tot_cur_bal', 'open_acc_6m'

<br>
<br>
Let's categorize loans into two categories and see how many examples fall into each one of them. I'll assign 'bad loans' as ones and 'good loans' as zeros:

In [461]:
in_interest = ['Charged Off', 
               'Does not meet the credit policy. Status:Charged Off', 
               'Late (31-120 days)', 
               'Default']

df['labels'] = np.where(df['loan_status'].isin(in_interest), 1.0, 0.0)
df['labels'].value_counts(normalize=True)

0.0    0.93306
1.0    0.06694
Name: labels, dtype: float64

<br>
<br>
Essentialy we're now facing binary classification problem with seriously unbalanced data. We may have to deal with this disproportion later on due to overfitting by applying regularization or data augmentation techniques, but let's first see how our models deal with data as they are.

First we need to clean our data and deal with NaN values. Let's see how many NaNs each of our features contain:

In [462]:
df.isnull().sum().sort_values(ascending=False) / len(df)

annual_inc_joint               0.99954
verification_status_joint      0.99954
dti_joint                      0.99954
il_util                        0.97954
mths_since_rcnt_il             0.97718
inq_fi                         0.97652
inq_last_12m                   0.97652
all_util                       0.97652
max_bal_bc                     0.97652
open_rv_24m                    0.97652
total_cu_tl                    0.97652
total_bal_il                   0.97652
open_il_24m                    0.97652
open_il_12m                    0.97652
open_il_6m                     0.97652
open_acc_6m                    0.97652
open_rv_12m                    0.97652
desc                           0.85326
mths_since_last_record         0.84592
mths_since_last_major_derog    0.75070
mths_since_last_delinq         0.51432
next_pymnt_d                   0.28720
tot_cur_bal                    0.08044
tot_coll_amt                   0.08044
total_rev_hi_lim               0.08044
emp_title                

At first glance some of these features are completely useless for us. There seem to be big leap between <b>next_pymnt_d</b> and <b>total_rev_hi_lim</b>, so we may choose 0.1 threshold, noting that this decision is arbitrary and we may revisit this later on.

Let's drop all columns that exceed our threshold and check with how many we're left: 

In [463]:
threshold = 0.1  

mask = df.isnull().sum()/len(df) <= threshold
print('Number of dropped columns: ', mask.value_counts()[False])

df = df[df.columns[mask]]

Number of dropped columns:  22


Let's now eliminate every left feature that won't provide any useful information given our task. I'm gonna approach this very cautiously due to my lack of expertise on the subject of loans and delete only obviously redundant columns:

In [464]:
redundant_columns = ['loan_status', 'id', 'zip_code', 'policy_code', 'title', 'url', 'grade']
df = df.drop(redundant_columns, axis=1)
# I've dropped 'grade' category since 'sub_grade' contain all that information already. The rest, I think, is self-explanatory

We're still left with 47 features. Some of them are categorical, so first let's find out which ones:

In [465]:
df.select_dtypes('object').apply(pd.Series.nunique, axis = 0)

term                       2
sub_grade                 35
emp_title              26311
emp_length                11
home_ownership             5
verification_status        3
issue_d                  102
pymnt_plan                 2
purpose                   14
addr_state                49
earliest_cr_line         595
initial_list_status        2
last_pymnt_d              94
last_credit_pull_d        89
application_type           2
dtype: int64

<br>

Upon further inspection (which I won't be presenting here since it would take too much space) I found out that:

<b>pymnt_plan</b> and <b>application_type</b> can be deleted, because the first one has only one value and the second one has 9996:4 disproportion between labels. Also <b>member_id</b> can be dropped since there are no members who took more than one loan.

<b>issue_d, earliest_cr_line, last_pymnt_d, last_credit_pull_d </b>are dates. They have too many unique values to represent them as one-hot vectors (too big dimensionality for my computer). I might put them in fewer time period brackets (eg. by years), but we would lose a lot of information that way. I decided to use following heuristic: encode dates as unix-time format. It can be beneficial in case I will be using any distance-based methods further down the line. 

<b>addr_state, sub_grade, emp_length, home_ownership, verification_status</b> and purpose all have less than 50 unique values, I can encode them as one-hots or process with sclearn LabelEncoder.

<b>emp_title</b> has 6413 unique values, but only 141 of them appear more than 5 times, and only 50 more than 10. I will use the latter as cut-off and put all titles which appeared less than 50 times in 'Other' category. If during evaluation of feature importance I will find out that employment title is important, I will investigate this category further. 

So, here's the code for discrabed steps:

In [466]:
# DELETE PYMNT_PLAN, APPLICATION_TYPE AND MEMBER ID

df = df.drop(columns=['pymnt_plan','application_type', 'member_id'])

In [467]:
# DATES

def _convert_string_to_unix_timestamp(date):
    dt = datetime.strptime(date, '%b-%Y')
    epoch = datetime(1970, 1, 1) 
    diff = dt - epoch
    # I need to use this workaround since mktime doesn't support dates from before the Unix epoch (1970) on Windows
    try:
        timestamp = time.mktime(dt.timetuple())
    except OverflowError:
        timestamp = diff.total_seconds()
    
    return timestamp


date_columns = ['issue_d', 'earliest_cr_line', 'last_pymnt_d', 'last_credit_pull_d']

# Replace NaN values with value of most occuring record. We can also drop all records containing NaNs, but for the time being
# I chose first method.
most_occuring_value = df[date_columns].apply(pd.Series.value_counts).sum(axis=1).idxmax()
df[date_columns] = df[date_columns].fillna(most_occuring_value)

df[date_columns] = df[date_columns].applymap(_convert_string_to_unix_timestamp)

In [468]:
# REPLACE EMP_TITLE VALUES OCCURING LESS THAN 10 TIMES WITH 'OTHER'

df['emp_title'] = df['emp_title'].fillna('NaN')  # Replace NaN with string value
count_dict = df['emp_title'].value_counts().to_dict()
df['emp_title'] = df['emp_title'].apply(lambda row: row if count_dict[row] > 10 else 'Other')

In [469]:
# CREATE ONE-HOT REPRESENTATIONS OF ALL CATEGORICAL VALUES

categorical_columns = df.select_dtypes(include=["object"]).columns
df = pd.get_dummies(df, columns=categorical_columns, dtype=np.float64)

In [470]:
# TAKE CARE OF REMAINING NAN (IN COLUMNS THAT HAD TYPE FLOAT FROM THE BEGGINING)
df = df.fillna(df.mean())

In [471]:
# GET FEATURE MATRIX
labels = df['labels'].values
df = df.drop(columns=['labels'])
features = df.values
features.shape

(50000, 528)

Our feature matrix has shape (10000, 527). Let's break our dataset into train and test sets.

In [472]:
# Shuffle
dataset = np.concatenate((features, labels.reshape(features.shape[0], 1)), axis=1)
np.random.shuffle(dataset)

# Split into train and test, no validation set since specification of this task doesn't mention it.
split = int(dataset.shape[0] * 0.3)
train_set, test_set = dataset[split:], dataset[:split]

# Split into features and labels 
X_train, Y_train = train_set[:,:-1], train_set[:, -1]
X_test, Y_test = test_set[:,:-1], test_set[:, -1]

<br><br><font size="10"> Models </font><br><br>

We can now start training our models.

For baseline method I'm going to be using logistic regression:

In [473]:
from sklearn.linear_model import LogisticRegression

log_reg = LogisticRegression()
log_reg.fit(X_train, Y_train)

C:\Users\Kamil\Anaconda\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

Let's see the scores! 

In [474]:
from sklearn.metrics import accuracy_score, recall_score, f1_score, precision_score

predictions_log = log_reg.predict(X_test)
print(' Accuracy: ', accuracy_score(Y_test, predictions_log), '\n',
      'Precision: ', precision_score(Y_test, predictions_log), '\n',
      'Recall: ', recall_score(Y_test, predictions_log), '\n',
      'F1: ', f1_score(Y_test, predictions_log))

 Accuracy:  0.9293333333333333 
 Precision:  0.4375 
 Recall:  0.03333333333333333 
 F1:  0.061946902654867256


The scores seem to be awful. High accuracy means almost nothing to us, recall is the score we should be trying to maximize since False Negatives are the biggest issue for us (detecting bad loans). As predicted we face a serious case of overfitting.

We can now test other classification algorithms (k-nn, svm, xgboost, neural networks, lightGBM etc.), but first I would like to focus more on feature selection and reducing features dimentionality, since this may lead to reducing overfitting, especialy when our feature space is rather bloated right now.

I'm going to use tree-based feature selection.

In [475]:
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.feature_selection import SelectFromModel

print(X_train.shape)
clf = ExtraTreesClassifier(n_estimators=50)
clf = clf.fit(X_train, Y_train)
model = SelectFromModel(clf, prefit=True)
X_train_new = model.transform(X_train)

X_train_new.shape               

(35000, 528)


(35000, 79)

Ok, we've managed to reduce our features number, which not only will reduce overfitting, but will be handy in further iterations of data analysis! 

Currently I would like to get back to exploring the data and selected features, starting with the variables that were selected by tree-based method as most important. Only after that I would explore different models. Unfortunetly due to time constrictions I need to skip this step.

Let's see how well the model performed.

In [476]:
predictions_clf = clf.predict(X_test)
print(' Accuracy: ', accuracy_score(Y_test, predictions_clf), '\n',
      'Precision: ', precision_score(Y_test, predictions_clf), '\n',
      'Recall: ', recall_score(Y_test, predictions_clf), '\n',
      'F1: ', f1_score(Y_test, predictions_clf))

 Accuracy:  0.9677333333333333 
 Precision:  0.9982394366197183 
 Recall:  0.54 
 F1:  0.7008652657601978


Better, but still recall is not high enough for the model to be useful. Higher precision is good (we're not labeling good loans as bad), but we're still not detecting bad loans good enough. 

Let's try out more powerful model (to be precise, ensemble of models).

In [477]:
from xgboost import XGBClassifier

xgb = XGBClassifier()
xgb = xgb.fit(X_train, Y_train)

In [478]:
predictions_xgb = xgb.predict(X_test)
print(' Accuracy: ', accuracy_score(Y_test, predictions_xgb), '\n',
      'Precision: ', precision_score(Y_test, predictions_xgb), '\n',
      'Recall: ', recall_score(Y_test, predictions_xgb), '\n',
      'F1: ', f1_score(Y_test, predictions_xgb))

 Accuracy:  0.9950666666666667 
 Precision:  0.9969450101832994 
 Recall:  0.9323809523809524 
 F1:  0.9635826771653544


Unfortunetly that is all the time I can dedicate towards this task. If I was given more of it, I would probably approach this problem further in this way:

1. As mentioned before, feature selection seem to be interesting and important task. It would require me to revisit data and see which features are important, see with which variables they correlate and which specific examples models are unable to detect.

2. After further research I would attempt training ensambles of models and test them more rigorously (via k-fold validation, plotting ROC curves against precision-recall curves). I would probably try to use regularization (L1, L2, data augmentation, random noise etc.).

3. I would test different algorithms (mainly Neural Networks) and attempt tuning hyperparameters via grid-search.

4. Plot some graphs! :)